In [ ]:
import sys
sys.path.insert(1, '/home/fishial/Fishial/Object-Detection-Model')
import argparse
import cv2
import os
import random

import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

import yaml
import torch
from PIL import Image
from torchvision import transforms
from torch import nn
from tqdm import tqdm

import fiftyone as fo
import fiftyone.zoo as foz
from module.classification_package.src.utils import read_json, save_json
from module.classification_package.src.dataset import FishialDatasetFoOnlineCuting
from module.classification_package.src.model import init_model
from module.segmentation_package.src.utils import get_mask

import torchvision.models as models


In [ ]:
def get_image(img_path, polyline):
    full_image = cv2.imread(img_path)
    mask_np = get_mask(full_image, np.array(polyline))
    mask_pil = Image.fromarray(mask_np)
    mask_tensor = loader(mask_pil)
    
    return mask_pil, mask_tensor.unsqueeze(0) # add batch dimension

In [ ]:
loader = transforms.Compose([
        transforms.Resize((224, 224), Image.BILINEAR),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

In [ ]:
EPXPORT_PATH = r'/home/fishial/Fishial/dataset/export_07_09_2023/06_export_Verified_ALL.json'
# data_export = read_json(EPXPORT_PATH)

# category_to_species_id = {data['supercategory']: data['fishial_extra']['species_id']
#     for data in data_export['categories'] if data['name'] == 'General body shape' 
# }

Labels have to be like this: 

```json
{"0": {"label_name": "Sciaenops ocellatus", "species_id": "92971e92-c01b-4f19-8189-482eac40ceac"},
 "1": {"label_name": "Centropomus undecimalis", "species_id": "5c2ec367-99ec-4e67-9edd-735eeff36157"},
 ...
}
```

In [ ]:
MODEL_PATH = r'/home/fishial/Fishial/output/classification/resnet_18_triplet_08_09_2023_v06_under_train_cross/model.ts'
model = torch.jit.load(MODEL_PATH)
model.eval()

labels = read_json('/home/fishial/Fishial/output/classification/resnet_18_triplet_08_09_2023_v06_under_train_cross/categories.json')

In [ ]:
labels = {
    labels[idx]['label_name']: {
        'internal_id': int(idx),
        'species_id': labels[idx]['species_id']
    } for idx in labels}


In [ ]:
fo_dataset = fo.load_dataset("classification-05-09-2023-v06")
fo_dataset = fo_dataset.match_tags(['val', 'train'])

### Let's create the list of description's of each elements which correspond of position in embeddig tensor list.

the discription have to be in next shape:

internal_id, image_id, annotation_id, drawn_fish_id

In [ ]:
data_set_ids = []
embedding_tensor = []

pbar = tqdm(fo_dataset)

for sample in pbar:
    img_path = sample['filepath']
    label = sample['polyline']['label']
    image_id, annotation_id, drawn_fish_id = sample['image_id'], sample['annotation_id'], sample['drawn_fish_id']
    width, height = sample['width'], sample['height']
    
    polyline = sample['polyline']['points'][0]
    polyline = [[int(point[0] * width), int(point[1] * height)] for point in polyline]
    
    pil_image, input_tensor = get_image(img_path, polyline) 
    
    output = model(input_tensor)[0]
    embedding_tensor.append(output[0].detach())
    
    data_set_ids.append([labels[label]['internal_id'], image_id, annotation_id, drawn_fish_id])

In [ ]:
#define the root rirectorhe where will be located tensor file and json 
absolute_path = '/home/fishial/Fishial/output/classification/resnet_18_triplet_08_09_2023_v06_under_train_cross'

#converting dict keys, name to the string name 
id_to_name = {
    labels[label]['internal_id']:{
        'name': label,
        'species_id': labels[label]['species_id']
    } for label in labels}

#combaine categories and list of descriptions
final_data = {
    'categories': id_to_name,x
    'list_of_ids': data_set_ids
}
#stack List[Tensors(256)] to Tensor(N, 256)
data_set = torch.stack(embedding_tensor)
#Save all of them
torch.save(data_set, os.path.join(absolute_path, 'embeddings.pt'))
save_json(final_data, os.path.join(absolute_path, 'idx.json'))